### Импорт библиотек.

In [1]:
import pandas as pd
import pyperclip as clp
import requests
import re

from datetime import date
from time import sleep
from extra_modules.hh_funcs import get_hh_token

### Загрузка токена.

In [2]:
# Загрузим полученный на оф.сайте токен:
hh_token = get_hh_token()

### Определение функции для совершения запроса.

In [3]:
def get_vacancy_info(vacancy_id):
    '''
    Функция для получения информации о вакансии. В качестве аргумента получает ID вакансии.
    '''

    # Создаем заголовок для авторизации на HH с заранее полученным токеном приложения:
    header = {'Authorization' : f'Bearer {hh_token}'}
    # Формируем ссылку на основе ID вакансии:
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'

    # Делаем таймаут при обращении к серверу, чтобы нас не забанили:
    sleep(1)
    # Делаем запрос:
    response = requests.get(url, headers=header)
    
    # Следующая строка может понадобиться при отладке: 
    # response.raise_for_status()

    # Возвращаем результат сразу в json-формате:
    return response.json()

### Создание списка интересующих вакансий.

In [4]:
# Накидываем в строку ID нужных вакансий через любой разделитель:
vacancies_str = '89211781 88651643 88900995'

# Формируем список ID вакансий:
vacancy_list = re.findall('\d+', vacancies_str)

vacancy_list

['89211781', '88651643', '88900995']

### Запросы.

In [5]:
# Создаем список ответов сервера и записываем в него результаты наших запросов:
results = []
for vac_id in vacancy_list :
    result = get_vacancy_info(vac_id) # Смело можем делать запросы в цикле - таймаут зашит в саму функцию.
    results.append(result)

### Обработка результатов.

In [6]:
# Создаем пустой датафрейм, куда будем записывать нужную нам информацию:
df = pd.DataFrame(columns=['date', 'employer_name','vacancy_name', 'vacancy_url','employer_id', 'vacancy_id'])

# Получаем сегодняшнюю дату:
today = date.today().strftime('%Y-%m-%d')

# Добавим в датафрейм интересующие нас поля:
for i, res in enumerate(results) :
    vac_id = res.get('id')
    vac_name = res.get('name')
    empl_name = res.get('employer').get('name')[:60]
    empl_id = res.get('employer').get('id')
    vac_url = res.get('alternate_url')
    current_str = [today, empl_name, vac_name, vac_url, empl_id, vac_id]
    df.loc[i] = current_str

# Посмотрим на результат:
df[['date', 'employer_name','vacancy_name', 'vacancy_url','employer_id', 'vacancy_id']].style.hide(axis="index")

date,employer_name,vacancy_name,vacancy_url,employer_id,vacancy_id
2023-11-16,ГАЛА-ЦЕНТР,"Аналитик ТН ""Дача и Сад""",https://hh.ru/vacancy/89211781,1274827,89211781
2023-11-16,Медико-фармацевтическое объединение малого бизнеса,Аналитик в отдел по работе с производителями (фарм.рынок),https://hh.ru/vacancy/88651643,2060223,88651643
2023-11-16,Самокат (ООО Умный ритейл),Бизнес-аналитик отдела продаж,https://hh.ru/vacancy/88900995,2460946,88900995


### Создание драфтов сопроводительных писем для каждой из вакансий.

In [7]:
# Загрузка драфта сопроводительного письма:
with open('res/cv_text.txt', 'r', encoding='utf-8') as f :
    body_txt = f.read()

body_txt = body_txt.strip()

# Создание адресных сопроводительных писем и копирование их в буфер обмена:
for company in df.employer_name.to_list() : 
    # Вставляем в шаблон название конкретной компании:
    text = body_txt.format(company_name=company)
    # Копируем в буфер обмена:
    clp.copy(text)
    # Время ожидания подобрано эмпирически, если его уменьшить, буфер не успевает среагировать:
    sleep(0.25)